In [1]:
import importlib
import pandas
import numpy
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

#### ЗАВАНТАЖЕННЯ ФАЙЛУ З BIPRO.PROZORRO ІЗ ЗАВЕРШЕНИМИ ЗАКУПІВЛЯМИ МІСЬКИХ РАД.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Дата - від 1 серпня 2016 до 31 грудня 2019. 
"ИдентификаторЛота" є унікальним ідентифікатором рядка.¶

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep=';', low_memory = False)

In [4]:
training_data_columns = df_raw.columns
training_data_columns = training_data_columns.to_list()

#### Крок 0. Створити базу шкіл і садків, які є замовниками за базою МОН (positives)

In [5]:
# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_independent = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])
print('df_independent:', df_independent.shape)

# Створити базу для закупівель навчальних закладів, які НЕ є замовниками
df_merged_left = pandas.merge(df_raw, df_independent, on='ИдентификаторЛота', 
                              how='left', indicator=True, suffixes=('','_temp'))
df_undefined = df_merged_left[df_merged_left._merge=='left_only']
df_undefined = df_undefined[training_data_columns]

/home/artem/.local/lib/python3.6/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


537893 rows. After filtering: 38817
df_independent: (38817, 11)


In [6]:
# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']
df_step0 = df_undefined.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_independent_2, df_undefined_2 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "positives:", df_independent_2.shape, "negatives:", df_undefined_2.shape)

step0: positives: (12598, 12) negatives: (486478, 12)


#### Крок 1. Пошук за ключовими словами закупівель для навчальних закладів, які не є замовниками (negatives)

In [11]:
df_undefined_2

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Организатор_step0_check
0,UA-2016-07-11-000419-b-L1,UA-2016-07-11-000419-b,"Комунальне некомерційне підприємство ""Центр пе...",Скраплений газ,80987.9,NaN,Скраплений газ,"вул.Чумаченка,21-б, вул Космічна,125","вулиця Чумаченка, буд.21",Запорізька міська рада,38969678,False
1,UA-2016-07-11-000419-b-L1,UA-2016-07-11-000419-b,"Комунальне некомерційне підприємство ""Центр пе...",Скраплений газ,80987.9,NaN,Скраплений газ,"вул.Чумаченка,21-б, вул.Космічна,125","вулиця Чумаченка, буд.21",Запорізька міська рада,38969678,False
2,UA-2016-09-13-000354-a-L1,UA-2016-09-13-000354-a,ДЮСШ № 23,Придбання обладнання для медичного кабінету по...,17000,Обладнання для медичного кабінету позашкільног...,Придбання обладнання для медичного кабінету по...,"просп. Маяковського, 26-А","просп. Маяковського, 26-А",КМДА,34290400,False
3,UA-2016-11-10-001404-b-L1,UA-2016-11-10-001404-b,Управління освіти департаменту гуманітарної по...,Додаткова угода №1 до договору №1 від 30.06.20...,129257,Додаткова угода №1 до договору №1 від 30.06.20...,Додаткова угода №1 до договору №1 від 30.06.20...,"пл. Ринок,9","пл. Ринок, 9",Львівська міська рада,2144743,False
4,UA-2016-08-29-000321-b-L1,UA-2016-08-29-000321-b,Департамент культури виконавчого органу Київ...,Культурно-мистецький проект з організації та п...,15000,Відповідно до наказу Департаменту культури вик...,Культурно-мистецький проект з організації та п...,за адресою замовника,"бульвар Тараса Шевченка,3",КМДА,2231933,False
...,...,...,...,...,...,...,...,...,...,...,...,...
538578,UA-2019-04-19-000216-a-L1,UA-2019-04-19-000216-a,Управління освіти та інноваційного розвитку Пе...,ДК 021:2015 90910000-9 Послуги з прибирання (П...,49000,ДК 021:2015 90910000-9 Послуги з прибирання (П...,ДК 021:2015 90910000-9 Послуги з прибирання (П...,"вул. Московська, 39-А","вул. Інститутська, буд. 24/7",КМДА,39833860,False
538579,UA-2019-04-15-000644-c-L1,UA-2019-04-15-000644-c,Управління освіти та інноваційного розвитку Пе...,Поточний ремонт внутрішніх приміщень ЗДО № 14...,180000,Поточний ремонт внутрішніх приміщень ЗДО № 14...,Поточний ремонт внутрішніх приміщень ЗДО № 14...,"вул. Лесі Українки, 24-а","вул. Інститутська, буд. 24/7",КМДА,39833860,False
538580,UA-2019-04-19-000461-b-L1,UA-2019-04-19-000461-b,"Комунальне некомерційне підприємство ""Київська...",Лікарські засоби різні (лабораторні матеріали),19600,NaN,Лікарські засоби різні (лабораторні матеріали),"Шевченківський район, ВУЛИЦЯ ДЕГТЯРІВСЬКА, буд...","вул. Дегтярівська,23",КМДА,1993664,False
538581,UA-2019-04-03-001059-a-L1,UA-2019-04-03-001059-a,"КНП ""ПЦ М.КИЄВА""",Лікарські засоби (Лабораторні реактиви),28500,NaN,Лікарські засоби (Лабораторні реактиви),"вулиця Предславинська,9","Печерський район, ВУЛ. ПРЕДСЛАВИНСЬКА, будинок 9",КМДА,22964365,False


In [7]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'Организатор', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_2.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_step1_positives, df_step1_negatives = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_step1_positives = df_step1_positives[training_data_columns]
df_step1_negatives = df_step1_negatives[training_data_columns]
print(f"{step_mark}:", "positives:", df_step1_positives.shape, "negatives:", df_step1_negatives.shape)

step1: positives: (20218, 11) negatives: (466260, 11)


#### Крок 2. За ЄДРПОУ видалити закупівлі навчальних закладів, які не є закладами загальної середньої та дошкільної освіти

In [ ]:
# ЗАСТЕРЕЖЕННЯ: До переліку "негативних" ЄДРПОУ включені управління освіти (база МОН не ідеальна).
# ДОРОБИТИ: Заклади, де є "управління освіти" в полі організатор, не видаляти

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['IDOrganizator'].unique().tolist()

In [ ]:
filter_dataframe_inspect_before_drop_step_0['Организатор'].unique().tolist()

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [8]:
df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])
print('Залишилось закупівель навчальних закладів, які не є замовниками:', df_step2_positives.shape[0])

Dropped rows: 593
Dropped rows: 44


#### Крок 3. Об'єднання двох баз закупівель навчальних закладів, які є самостійними замовниками. Класифікація закладу для цієї бази - через колонку 'Организатор'

In [12]:
frames_independent = [df_independent, df_independent_2]
df_independent_concatenated = pandas.concat(frames_independent)
print(df_independent_concatenated.shape)
df_independent_concatenated.drop_duplicates(subset='ИдентификаторЛота', inplace = True)
print(df_independent_concatenated.shape)

(51415, 12)
(51062, 12)


In [13]:
df_step2_positives.columns

Index(['ИдентификаторЛота', 'Идентификатор', 'Организатор', 'Тендер',
       'СуммаЛота', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'IDOrganizator'],
      dtype='object')

#### Крок 4. Класифікація закладів, які не є самостійними замовниками

In [76]:
step_mark = 'step4'
action_mark = 'classification'
columns_to_classify_step4 = ['Организатор', 'Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step4 = df_step2_positives.copy()

""" TEMP """
df_step4 = df_step4[['Организатор', 'Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']][0:2000]

df_step4['Тип закладу'] = numpy.full((df_step4.shape[0],), filter_inshe)

def step4_classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + 
                 action_mark] = given_df[column].str.contains('|'.join(filter_list),
                                                              case = False, 
                                                              na = False, 
                                                              regex = True)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break
#     for column in columns_to_classify:
#         given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

for idx in range(len(filter_priority_new)):
    df_step4 = step4_classification(df_step4, columns_to_classify_step4, 
                                    filter_priority[idx], filter_name_priority[idx])

df_step4['Тип закладу'].value_counts()

ДНЗ                                         696
Інше                                        666
Загальноосвітня школа                       350
Гімназія                                     96
Спеціалізована школа                         81
Навчально-виховний комплекс (об'єднання)     67
Ліцей                                        29
Інтернат                                     14
Спеціальна загальноосвітня школа              1
Name: Тип закладу, dtype: int64

## ПРОБЛЕМНО КЛАСИФІКУЄ "ІНШЕ" 

In [51]:
pandas.set_option('max_colwidth', 200)
df_step4[df_step4['Тип закладу'] == "Інше"]

,Организатор,Тендер,ОписаниеТендера,Лот,Адрес поставки,Тип закладу,Организатор_step4_classification,Тендер_step4_classification,ОписаниеТендера_step4_classification,Лот_step4_classification,Адрес поставки_step4_classification
159,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,"вул.Єреванська, 20",Інше,False,False,False,False,False
186,Управління у справах фізичної культури і спорту Миколаївської міської ради,"поточнийремонтдушових приміщень спортивної школи розташованої за адресою: пл. Заводська, 1",додаток до річного плану,"поточнийремонтдушових приміщень спортивної школи розташованої за адресою: пл. Заводська, 1","вул. Потьомкінська,95-А",Інше,False,False,False,False,False
248,Управління у справах фізичної культури і спорту Миколаївської міської ради,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А",додаток до річного плану,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А","вул. Потьомкінська,95-А",Інше,False,False,False,False,False
258,Департамент житлово-комунального господарства Чернівецької міської ради,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,NaN,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,Героїв Майдану 176,Інше,False,False,False,False,False
286,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний ремонт котельні дитячого садку №211,Поточний ремонт котельні дитячого садку №211,Поточний ремонт котельні дитячого садку №211,"Вул. Леніна, 167",Інше,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
34598,Департамент освіти і науки Запорізької міської ради,Завершальні будівельні роботи (Поточний ремонт по заміні віконних блоків у приміщенні ЗОШ№ 85 м.Запоріжжя,NaN,Завершальні будівельні роботи (Поточний ремонт по заміні віконних блоків у приміщенні ЗОШ№ 85 м.Запоріжжя,пров.Економічний 5,Інше,False,False,False,False,False
34632,Управління освіти Херсонської міської ради,капітальний ремонт (заміна вікон)в ЗОШ № 4,NaN,капітальний ремонт (заміна вікон)в ЗОШ № 4,"пров. Комбайновий, 13",Інше,False,False,False,False,False
34635,УПРАВЛІННЯ ОСВІТИ КРАМАТОРСЬКОЇ МІСЬКОЇ РАДИ,"Виконання технічного нагляду по об'єкту ""Капітальний ремонт актової зали ЗОШ № 22 за адресою :Донецька обл. м. Краматорськ, вул. Двірцева,3""",NaN,"Виконання технічного нагляду по об'єкту ""Капітальний ремонт актової зали ЗОШ № 22 за адресою :Донецька обл. м. Краматорськ, вул. Двірцева,3""","вул. Ярослава Мудрого,56",Інше,False,False,False,False,False
34671,"Департамент у справах сім'ї, молоді та спорту Харківської міської ради","Послуги з оренди апаратури для проведення заходу в рамках проведення ""Школи волонтерів"" з 8 по 22 вересня 2017 року","Послуги з оренди апаратури для проведення заходу в рамках проведення ""Школи волонтерів"" з 8 по 22 вересня 2017 року: екран-проектор.","Послуги з оренди апаратури для проведення заходу в рамках проведення ""Школи волонтерів"" з 8 по 22 вересня 2017 року","вул. Ковальська, 9",Інше,False,False,False,False,False


#### Крок 5. Класифікація навчальних закладів, які є замовниками

In [25]:
step_mark = 'step5'
action_mark = 'classification'
columns_to_classify_step5 = ['Организатор']
df_step5 = df_independent_concatenated.copy()

In [26]:
def step5_classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), case = False, na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [27]:
for idx in range(len(filter_priority)):
    df_step5 = step5_classification(df_step5, columns_to_classify_step5, filter_priority[idx], filter_name_priority[idx])

In [28]:
df_step5['Тип закладу'].value_counts()

ДНЗ                                          11057
Загальноосвітня школа                         5719
Спеціалізована школа                          5321
Ліцей                                         4964
Навчально-виховний комплекс (об'єднання)      2003
Гімназія                                      1443
Інтернат                                       795
Спеціальна загальноосвітня школа               245
Міжшкільний навчально-виробничий комбінат        5
Name: Тип закладу, dtype: int64

In [ ]:
Крок 6.